- try and correct label smoothing (0.00005)
- add umap
- 3 ensemble (5 folds) 

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold
from torch.nn.modules.loss import _WeightedLoss

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[~train.index.isin(noncons_train_index)].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
#X = train[g_feats+c_feats].copy().values
#select = VarianceThreshold(threshold=0.4)
#X_new = select.fit_transform(X)
#drop_feats = list(np.array(train[g_feats+c_feats].columns)[select.get_support()==False])
#len(drop_feats)

# feature engineering

In [7]:
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [8]:
c_num = 1
pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
pca = PCA(n_components=c_num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

g_num = 10
pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
pca = PCA(n_components=g_num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [9]:
uc_num = 1
um = UMAP(n_neighbors=30, n_components=uc_num, random_state=42)
um_c_cols = ["um-c"+str(i+1) for i in range(uc_num)]
uc_train = um.fit_transform(train[c_feats])
uc_test = um.transform(test[c_feats])
uc_train = pd.DataFrame(uc_train, columns=um_c_cols)
uc_test = pd.DataFrame(uc_test, columns=um_c_cols)

ug_num = 10
um = UMAP(n_neighbors=30, n_components=ug_num, random_state=42)
um_g_cols = ["um-g"+str(i+1) for i in range(ug_num)]
ug_train = um.fit_transform(train[g_feats])
ug_test = um.transform(test[g_feats])
ug_train = pd.DataFrame(ug_train, columns=um_g_cols)
ug_test = pd.DataFrame(ug_test, columns=um_g_cols)

train = pd.concat([train, uc_train],axis=1)
test = pd.concat([test, uc_test],axis=1)
train = pd.concat([train, ug_train],axis=1)
test = pd.concat([test, ug_test],axis=1)

In [10]:
#n_clusters_g = 15
#n_clusters_c = 5
#def create_cluster(train, test, kind, n_clusters):
#    if kind == "g":
#        train_ = train[pca_g_cols].copy()
#        test_ = test[pca_g_cols].copy()
#    else:
#        train_ = train[pca_c_cols].copy()
#        test_ = test[pca_c_cols].copy()    
#    kmeans = KMeans(n_clusters = n_clusters, random_state = 0).fit(train_)
#    train[f'clusters_{kind}'] = kmeans.labels_
#    test[f'clusters_{kind}'] = kmeans.predict(test_)
#    train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
#    test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
#    return train, test
    
#train, test = create_cluster(train, test, kind = 'g', n_clusters = n_clusters_g)
#train, test = create_cluster(train, test, kind = 'c', n_clusters = n_clusters_c)

In [11]:
def fe(df):
    tmp = df.copy()
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})      
    tmp.drop(["sig_id", "cp_type"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 896) (3982, 896)


In [12]:
#train.drop(drop_feats, axis=1, inplace=True)
#test.drop(drop_feats, axis=1, inplace=True)

In [13]:
train.head()

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,um-g1,um-g2,um-g3,um-g4,um-g5,um-g6,um-g7,um-g8,um-g9,um-g10
0,24,0,1.111801,0.903367,-0.433829,-0.971728,-0.286559,-1.011388,-1.357431,-0.041716,...,0.889615,1.461260,9.432432,5.493209,8.951550,-0.854605,9.944616,-0.028288,5.590945,-0.488484
1,72,0,0.105667,0.672509,0.257486,0.086759,1.199685,0.691813,0.353695,0.558374,...,1.124779,2.489649,9.759426,5.647223,9.838367,0.831224,8.591188,1.179337,5.231938,-0.486060
2,48,0,0.767036,0.942499,1.408911,-0.126492,-0.028694,1.490985,0.272541,0.359490,...,2.509898,1.760835,10.155374,4.504743,8.086551,-0.042348,7.571811,1.003149,5.557558,0.070003
3,48,0,-0.755626,-0.297077,-0.455058,0.765972,2.343522,-0.852713,-2.316440,0.301512,...,3.685860,1.621680,9.671547,7.625240,8.735300,0.747822,9.019852,0.336911,4.433204,1.609635
4,72,1,-0.468806,-0.504196,0.956769,0.975864,1.447729,-0.863807,-0.346926,-0.227072,...,1.506485,2.646500,9.428581,4.995689,9.087056,-0.205562,8.078624,0.751063,6.604461,-1.283124


In [14]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()
    
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [15]:
fn_train.shape, fn_test.shape, fn_targets.shape

((21948, 896), (3982, 896), (21948, 206))

# modelling

In [16]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(device) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss

In [17]:
smoothing = 0.00005
p_min = smoothing
p_max = 1 - smoothing

class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):

        logits = 1 / (1 + np.exp(-y_pred))
        logits = np.clip(logits, p_min, p_max)
        
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [18]:
MAX_EPOCH=200
device = "cuda" if torch.cuda.is_available() else "cpu"

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def modelling_tabnet(tr, target, te, sample_seed):
    seed_everything(sample_seed) 
    tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.5, seed = sample_seed,
                     lambda_sparse=0, n_independent=1, n_shared=1, 
                     optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []

    NB_SPLITS = 5
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = np.zeros([len(tr),target.shape[1]])
    scores = []
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
    
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=SmoothCrossEntropyLoss(smoothing=smoothing)) #torch.nn.functional.binary_cross_entropy_with_logits)
    
        preds_val = model.predict(X_val)
        # Apply sigmoid to the predictions
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        #oof_preds[val_idx,:] = preds
        oof_preds[val_idx,:] = np.clip(preds, p_min, p_max)
        scores.append(score)

        # preds on test
        preds_test = model.predict(te)
        #test_cv_preds.append(1 / (1 + np.exp(-preds_test)))
        test_cv_preds.append(np.clip(1 / (1 + np.exp(-preds_test)), p_min, p_max))
        
    test_preds_all = np.stack(test_cv_preds)
    print("OOF log loss:", log_loss(np.ravel(target), np.ravel(np.array(oof_preds))))
    aucs = []
    for task_id in range(206):
        aucs.append(roc_auc_score(y_true=target[:, task_id],y_score=oof_preds[:, task_id]))
    print(f"Overall AUC : {np.mean(aucs)}")
    return oof_preds, test_preds_all

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [0,1,2]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(fn_train, fn_targets, fn_test, seed_)
    target_oof += oof_preds / len(seeds)
    target_pred += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 0.43658 | val_logits_ll: 0.14626 |  0:00:02s
epoch 10 | loss: 0.0192  | val_logits_ll: 0.01956 |  0:00:12s
epoch 20 | loss: 0.01717 | val_logits_ll: 0.01786 |  0:00:22s
epoch 30 | loss: 0.01643 | val_logits_ll: 0.01811 |  0:00:32s
epoch 40 | loss: 0.01585 | val_logits_ll: 0.01777 |  0:00:41s
epoch 50 | loss: 0.01579 | val_logits_ll: 0.01752 |  0:00:52s
epoch 60 | loss: 0.01517 | val_logits_ll: 0.01714 |  0:01:02s
epoch 70 | loss: 0.01503 | val_logits_ll: 0.01691 |  0:01:12s
epoch 80 | loss: 0.0147  | val_logits_ll: 0.01686 |  0:01:22s

Early stopping occured at epoch 87 with best_epoch = 67 and best_val_logits_ll = 0.01679
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 0.43536 | val_logits_ll: 0.15543 |  0:00:00s
epoch 10 | loss: 0.01948 | val_logits_ll: 0.01954 |  0:00:10s
epoch 20 | loss: 0.01738 | val_logits_ll: 0.01824 |  0:00:21s
epoch 30 | loss: 0.0168  | val_logits_ll: 0.0177  |  

In [19]:
aucs = []
for task_id in range(targets.shape[1]-1):
    aucs.append(roc_auc_score(y_true=targets.iloc[:, task_id+1].values,
                              y_score=target_oof[:, task_id]))
print(f"Overall AUC : {np.mean(aucs)}")

Overall AUC : 0.7667315284522015


In [20]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.015056983495328212


In [21]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)